# Compare TexAER Excel Output with TTI's Trends CSVs for QC  
Created by: Apoorb  
Created on: Aug 03, 2023

In [2]:
import pandas as pd
from pathlib import Path

## Load Data and Mapping Files

In [3]:
pa_dat = Path(r"C:\Users\a-bibeka\PycharmProjects\Loco-EI\data\processed\QC_TexAER_Aug2023")
pa_crosswalk_tti_texAER = pa_dat.joinpath("crosswalk_tti_texAER.xlsx")
x1 = pd.ExcelFile(pa_crosswalk_tti_texAER)
pa_cntr_emis = pa_dat.joinpath("Task6.2_Appendix G cntr_emis_quant_2011_50.csv")
pa_uncntr_emis = pa_dat.joinpath("Task6.2_Appendix G uncntr_emis_quant_2011_50.csv")

In [8]:
crosswalk_cnty = x1.parse("crosswalk_cnty")
crosswalk_cnty = crosswalk_cnty.drop(columns="TexAER_idx")
pol_ord = x1.parse("pol_ord")
pol_ord1 = list(pol_ord.pol_ord.astype(str))
o3d_pol = x1.parse("o3d_pol")
years = range(2025, 2034)
cntr_emis = pd.read_csv(pa_cntr_emis, index_col=0)
uncntr_emis = pd.read_csv(pa_uncntr_emis, index_col=0)
scc = pd.DataFrame({"scc": ['2285002006', '2285002007', '2285002008', '2285002009', '2285002010']})

C:\Users\a-bibeka\AppData\Local\Temp\ipykernel_24660\4287662209.py:8: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  uncntr_emis = pd.read_csv(pa_uncntr_emis, index_col=0)


## Reshape Data

Get the expected category combinations

In [10]:
exp_data_ = crosswalk_cnty.merge(scc, how="cross").merge(pol_ord, how="cross").rename(columns={"pol_ord": "pollutant", "TTI": "TTI_county", "TexAER": "County"})
exp_data_.loc[exp_data_.pollutant == "VOC"]

,County,TTI_county,scc,pollutant
0,ANDERSON COUNTY,Anderson,2285002006,VOC
53,ANDERSON COUNTY,Anderson,2285002007,VOC
106,ANDERSON COUNTY,Anderson,2285002008,VOC
159,ANDERSON COUNTY,Anderson,2285002009,VOC
212,ANDERSON COUNTY,Anderson,2285002010,VOC
...,...,...,...,...
67045,ZAVALA COUNTY,NaN,2285002006,VOC
67098,ZAVALA COUNTY,NaN,2285002007,VOC
67151,ZAVALA COUNTY,NaN,2285002008,VOC
67204,ZAVALA COUNTY,NaN,2285002009,VOC


Reshape the data

In [87]:
def piv_loco(exp_data_, emis_df, year_, emis_ton_col):
    emis_df_1 = emis_df.loc[lambda df: df.year == year_].groupby(['year', 'stcntyfips', 'county_name', 'scc', 'pollutant'])[emis_ton_col].sum().reset_index()
    emis_df_1["scc"] = emis_df_1["scc"].astype("Int64").astype(str)
    emis_df_1 = emis_df_1.rename(columns={"county_name": "TTI_county"})
    exp_data_["pollutant"] = exp_data_["pollutant"].astype(str)
    emis_df_1["pollutant"] = emis_df_1["pollutant"].astype(str)
    exp_data_1 = exp_data_.merge(emis_df_1, on=["TTI_county", "scc", "pollutant"], how="left")
    exp_data_1["year"] = year_
    exp_data_2 = exp_data_1.drop(columns=["TTI_county", "stcntyfips"]).set_index(["year", "County", "scc", "pollutant"]).unstack()
    set(exp_data_2.columns).symmetric_difference(set(pol_ord1))
    exp_data_3 = exp_data_2.droplevel(0, axis=1).reset_index()
    exp_data_4 = exp_data_3.rename(columns={"year": "Year", "scc": "SCC"}).filter(items=["Year", "County", "SCC"] + pol_ord1)
    exp_data_5 = exp_data_4.set_index(["Year", "County", "SCC"])
    exp_data_5 = exp_data_5.round(2)
    reindex_idx = []
    for i in o3d_pol.O3D_pol.unique():
        reindex_idx.append((i, "Annual"))
        reindex_idx.append((i, "O3D"))
    exp_data_5_o3d = exp_data_5[o3d_pol.O3D_pol.unique()]
    exp_data_5_o3d.columns = [(i,"Annual") for i in exp_data_5_o3d.columns]
    multi_idx = pd.MultiIndex.from_tuples(reindex_idx, names=('Pollutants', 'Temporal Scope'))
    exp_data_5_o3d_1 = exp_data_5_o3d.reindex(multi_idx, axis="columns")
    idx = pd.IndexSlice
    exp_data_5_o3d_1.loc[:, idx[:, "O3D"]] = (exp_data_5_o3d_1.loc[:, idx[:, "Annual"]] / 365).round(4).values
    ann_cols = [i for i in exp_data_5.columns if i not in o3d_pol.O3D_pol.unique()]
    exp_data_5_ann = exp_data_5[ann_cols]
    col_reindex = [(i,"Annual") for i in exp_data_5_ann.columns]
    exp_data_5_ann.columns = col_reindex
    multi_idx_1 = pd.MultiIndex.from_tuples(col_reindex, names=('Pollutants', 'Temporal Scope'))
    exp_data_5_ann = exp_data_5_ann.reindex(multi_idx_1, axis="columns")
    exp_data_5_o3d_ann = pd.concat([exp_data_5_o3d_1, exp_data_5_ann], axis=1).fillna(0)
    return exp_data_5_o3d_ann

In [88]:
x_out = pd.ExcelWriter(pa_dat.joinpath(f"tti_csv_2025to2033_controlled_emissions.xlsx"))
for year in years:
    data = piv_loco(
        exp_data_=exp_data_, 
        emis_df=cntr_emis,
        year_=year,
        emis_ton_col="controlled_em_quant_ton"
    )
    data.to_excel(x_out, sheet_name=str(year))
x_out.close()

In [89]:
x_out = pd.ExcelWriter(pa_dat.joinpath(f"tti_csv_2025to2033_uncontrolled_emissions.xlsx"))
for year in years:
    data = piv_loco(
        exp_data_=exp_data_, 
        emis_df=uncntr_emis,
        year_=year,
        emis_ton_col="uncontrolled_em_quant_ton"
    )
    data.to_excel(x_out, sheet_name=str(year))
x_out.close()

In [90]:
x_in = pd.ExcelFile(pa_dat.joinpath(f"tti_csv_2025to2033_controlled_emissions.xlsx"))
tti_2025_cntr = x_in.parse("2025", index_col=[0,1,2], header=[0,1])

In [91]:
columns = tti_2025_cntr.reset_index().columns[1:]

In [92]:
pa_texAER_2025 = pa_dat.joinpath("Locomotive Trend EIs_CY2025 Controlled_TexAER.xlsx")
texAER_2025 = pd.read_excel(pa_texAER_2025, skiprows=10, header=None)
texAER_2025.columns = columns
texAER_2025 = texAER_2025.set_index(["County", "SCC"])

In [93]:
texAER_2025 = texAER_2025.drop("Total")

C:\Users\a-bibeka\AppData\Local\Temp\ipykernel_24660\2469050804.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  texAER_2025 = texAER_2025.drop("Total")


In [94]:
import numpy as  np

In [95]:
idx = pd.IndexSlice
test_1 = tti_2025_cntr.loc[:, idx["VOC", "O3D"]]
test_2 = texAER_2025.loc[:, idx["VOC", "O3D"]]

In [96]:
test_2.loc[~np.isclose((test_1 -test_2).values, 0)]

County            SCC         
ANDERSON COUNTY   2.285002e+09    0.0029
ANGELINA COUNTY   2.285002e+09    0.0003
BRAZORIA COUNTY   2.285002e+09    0.0214
CHAMBERS COUNTY   2.285002e+09    0.0020
CRANE COUNTY      2.285002e+09    0.0003
DALLAM COUNTY     2.285002e+09    0.0029
DELTA COUNTY      2.285002e+09    0.0000
GONZALES COUNTY   2.285002e+09    0.0069
GUADALUPE COUNTY  2.285002e+09    0.0006
HALE COUNTY       2.285002e+09    0.0031
HILL COUNTY       2.285002e+09    0.0142
HOCKLEY COUNTY    2.285002e+09    0.0009
KAUFMAN COUNTY    2.285002e+09    0.0077
MATAGORDA COUNTY  2.285002e+09    0.0000
MEDINA COUNTY     2.285002e+09    0.0008
MILAM COUNTY      2.285002e+09    0.0328
                  2.285002e+09    0.0000
MILLS COUNTY      2.285002e+09    0.0000
MONTAGUE COUNTY   2.285002e+09    0.0100
ROBERTSON COUNTY  2.285002e+09    0.0029
ROCKWALL COUNTY   2.285002e+09    0.0011
SHELBY COUNTY     2.285002e+09    0.0023
STARR COUNTY      2.285002e+09    0.0014
TRINITY COUNTY    2.285002

In [100]:
test_1.loc[~np.isclose((test_1 -test_2).values, 0, atol=0.001)]

Series([], Name: (VOC, O3D), dtype: float64)

In [105]:
assert np.allclose(tti_2025_cntr.loc[:, idx[:, "Annual"]], texAER_2025.loc[:, idx[:, "Annual"]])

True

In [109]:
assert np.allclose(tti_2025_cntr.loc[:, idx[:, "O3D"]], texAER_2025.loc[:, idx[:, "O3D"]], atol=0.0001)